---
# Modelling hippocampal neurons of animals <br> navigating in VR with recurrent neural networks
### Marco P. Abrate, Daniel Liu &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; University College London (UCL)
---

#### Outline
**Part 1: Rat simulation in 3D**
- Motion model with `RatInABox`

- Environment design

- Simulated rat vision with `ratvision`

**Part 2: Vision autoencoder**

**Part 3: Hippocampus model with RNN**

**Part 4: Hidden state representations analysis**
- Rate maps

- Polar maps

- Quantitive metrics

- Applications

---
## **Part 4: Hidden state representations analysis**
In this notebook, we will write code to extract representations from the **RNN hidden states**. These includes:
- Rate maps
- Polar maps
- Quantitative metrics
- Applications

Before starting this notebook, make sure you have:
- trajectory data from part 1, including speed and rotational speed
- embedded vision data from the Vision Autoencoder we trained in part 2
- RNN model of the hippocampus we trained in part 3


### 0. Install and import dependencies

In [ ]:
!pip install torch torchvision torchaudio
!pip install numpy
!pip install matplotlib
!pip install scipy

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")
    
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

### 1. Load evaluation data and trained RNN

As a first step, we will import the definitions of the RNN we coded in part 3, and load the trained weights. We will also load the embeddings of the data we will use to evaluate the hidden state representations, and some auxiliary variables that we will use to compute the rate maps.

In [ ]:
from utils import create_multiple_subsampling, SensoryDataset, PredictiveRNN

In [ ]:
# let's load the test data and process it into a dataloader, just like we did in the last part.

STRIDE = 10
d = '../data/adult'
trial_paths = sorted([p for p in Path(d).iterdir() if 'exp' in p.name])

repr_embeddings = []
repr_vel, repr_rotvel, repr_pos, repr_hds = [], [], [], []

for idx in range(20, len(trial_paths)):
    tp = trial_paths[idx]
    repr_embeddings.append(
        create_multiple_subsampling(np.load(tp / 'vision_embeddings.npy'), stride=STRIDE)
    )
    repr_vel.append(
        create_multiple_subsampling(
            np.load(tp / 'riab_simulation' / 'velocities.npy'), stride=STRIDE, is_velocity=True
        )
    )
    repr_rotvel.append(
        create_multiple_subsampling(
            np.load(tp / 'riab_simulation' / 'rot_velocities.npy')[..., None],
            stride=STRIDE, is_velocity=True
        )
    )
    repr_pos.append(
        create_multiple_subsampling(np.load(tp / 'riab_simulation' / 'positions.npy'), stride=STRIDE)
    )
    repr_hds.append(
        create_multiple_subsampling(np.load(tp / 'riab_simulation' / 'thetas.npy')[..., None], stride=STRIDE)
    )

repr_embeddings = np.concatenate(repr_embeddings, axis=0)
repr_vel = np.concatenate(repr_vel, axis=0)
repr_rotvel = np.concatenate(repr_rotvel, axis=0)
repr_pos = np.concatenate(repr_pos, axis=0)
repr_hds = np.concatenate(repr_hds, axis=0)

dataloader = torch.utils.data.DataLoader(
    SensoryDataset(
        repr_embeddings, repr_vel, repr_rotvel, repr_pos, repr_hds
    ), shuffle=False
)

Let's re-define the `PredictiveRNN` model, just like we did in the last part, and load the trained weights.

In [ ]:
N_HIDDEN = 500

visual_embedding_dim = repr_embeddings.shape[-1]
motion_signal_dim = repr_vel.shape[-1] + repr_rotvel.shape[-1]
trained_rnn_weights = '../part3/rnn.pth'

rnn = PredictiveRNN(
    DEVICE,
    n_inputs = visual_embedding_dim + motion_signal_dim,
    n_hidden = N_HIDDEN,
    n_outputs = visual_embedding_dim
)

loss_fn = torch.nn.L1Loss()

# load the trained weights
rnn.load_state_dict(torch.load(trained_rnn_weights, weights_only=True, map_location=DEVICE))
rnn = rnn.to(DEVICE)

### 2. Get hidden states from trained RNN

In the previous part, we wrote a function named `evaluate_rnn()` that &mdash; when the parameter `for_ratemaps` is set to `True` &mdash; will return a dictionary containing the hidden staes, positions, head directions, velocities, etc. at each time-step for convenience of our analysis.

In [ ]:
from utils import evaluate_rnn

d = evaluate_rnn(DEVICE, rnn, dataloader, loss_fn, for_ratemaps=True)

hidden_states = d['hidden_states']
positions = d['positions']
head_directions = d['head_directions']

# Reshape the hidden states, positions, and head directions to
# flatten out the number of trials dimension and keep (timesteps, features)
hidden_states = hidden_states.reshape(-1, hidden_states.shape[-1])
positions = positions.reshape(-1, positions.shape[-1])
head_directions = head_directions.reshape(-1, head_directions.shape[-1])

print(
    'RNN loss (check it does not deviate too much from previous part figure):',
    np.round(np.mean(d['batch_losses']), 2)
)
print(
    'Hidden state, positions, and head directions shapes:\n',
    hidden_states.shape, positions.shape, head_directions.shape
)

### 3. Rate maps

The **rate map** of a neuron is the two-dimensional representation of firing rates (or activity in our case) across spatial locations. In the real brain, neuron firings are stochastic. This means there is a lot of noise in the process, and the average activity of each bin is a crude (but unbiased) estimate of the real tuning curve, depending on the amount of data provided. The use of rate maps is standard practice in neuroscience for analysing place cell recordings [1].

We divide the arena into a 25x25 grid (625 bins) and calculate the rate map R for each unit k by averaging unit activity within each spatial bin:

$$
    R_{i,j}^{(k)} = \frac{\sum_t A_t^{(k)} \cdot\overrightarrow{1}_{\mathbf{p}_t \in \text{Bin}_{(i,j)}}}{\sum_t \overrightarrow{1}_{\mathbf{p}_t \in \text{Bin}_{(i,j)}}}\quad i, j \in \{1, \dots, B_p\}\times \{1, \dots, B_p\}
$$

where $A^{(k)}$ is the activity of hidden unit $k$, $\mathbf{p}$ is the agent's position vector (x and y components), and $B_p$ is the number of bins along each direction.

Computationally, it requires to discretise the environment into smallers bins &mdash; see `bin_data()` &mdash; then compute the average activity at each bin &mdash; see `compute_rate_maps()`.

In [ ]:
from scipy.ndimage import gaussian_filter

def bin_data(data, n_bins, limits, weights=[]):
    """
    Creates an N-dimensional histogram of data using n_bins between limits.
    If weights is provided, it creates a weighted histogram.
    
    Args:
        data: The data to be binned of shape (n_samples, n_dim).
        n_bins: The number of bins for each dimension.
        limits (list of tuples): The lower and upper limits for each dimension.
        weights: 1D array-like, optional
            The weights for the counts in the histogram.
    
    Returns:
        binned_data: The N-dimensional histogram of data.
    """
    
    n_dim = data.shape[-1]
    if np.size(n_bins) < n_dim:
        n_bins = np.repeat(n_bins, n_dim)
    
    bins = []
    if n_dim == 1:
        bins.append(np.linspace(limits[0], limits[1], n_bins+1))
    else:
        for i in range(n_dim):
            bins.append(np.linspace(limits[i][0], limits[i][1], n_bins[i]+1))
        
    if len(weights) == 0:
        hst = np.histogramdd(data, bins=bins)
    else:
        hst = np.histogramdd(data, bins=bins, weights=weights, density=False)

    return hst[0]


def compute_rate_maps(
    hidden_states:np.array, pos:np.array, sigma:float,
    n_bins:int, limits=[(0, 0.635),(0, 0.635)]
):
    """
    Calculates the smooth rate map for each neuron based on the given hidden states and trajectory.

    Args:
        hidden_states (np.array): (timesteps, n_hidden) Array of hidden states for each neuron.
        pos (np.array): (timesteps, 2) Array of (x,y) positions.
        sigma (float): Standard deviation of the Gaussian filter.
        n_bins (float): Number of bins for the positions.
        limits (list): Lower and upper limits of the positions.

    Returns:
        rate_maps (np.array): Rate maps for each neuron of shape (n_hidden, n_bins, n_bins).
        occupancy (np.array): Occupancy map of shape (n_bins, n_bins).
    """
    
    occupancy = bin_data(pos, n_bins=n_bins, limits=limits)
    if sigma > 0:
        occupancy = gaussian_filter(occupancy,sigma)

    n_cells = hidden_states.shape[1]
    rate_maps = np.empty(shape=(n_cells,occupancy.shape[0],occupancy.shape[1]))
    for i in range(n_cells):
        # YOUR CODE HERE (1)
        # calculate the activations for each cell, for each binned position
        activations = None
        
        # YOUR CODE HERE (2)
        # obtain the rate map for each cell by dividing the activations by the occupancy
        rate_maps[i,:,:] = None
    
    # flip y and swap x and y coordinates so
    # that they appear correctly in the image
    rate_maps = np.flip(rate_maps, axis=-1)
    rate_maps = np.transpose(rate_maps, (0, 2, 1))

    return rate_maps, occupancy

Often, we make the assumption that a neuron's activity doesn't change too much in neighbouring bins. To make our rate maps look smoother, it is customary to apply **Gaussian smoothing** to the rate maps.

In [ ]:
N_BINS = 25

rate_maps, occupancy = compute_rate_maps(hidden_states, positions, n_bins=N_BINS, sigma=0)

rate_maps_smooth, occupancy_smooth = compute_rate_maps(hidden_states, positions, n_bins=N_BINS, sigma=1.)

print('Rate maps shape (n_hidden, n_bins, n_bins):', rate_maps_smooth.shape)

In [ ]:
idx = 9

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(rate_maps[idx], cmap='jet')
axs[0].set_title(f'Un-smoothed Rate Map for Neuron {idx}')
axs[0].set_xlabel('X bin')
axs[0].set_ylabel('Y bin')

axs[1].imshow(rate_maps_smooth[idx], cmap='jet')
axs[1].set_title(f'Smoothed Rate Map for Neuron {idx}')
axs[1].set_xlabel('X bin')
axs[1].set_ylabel('Y bin')

plt.show()

### 4. Polar maps

Comparable to rate maps, the **polar maps** show how a neuron fires when it is tuned to a specific angle in polar space by binnind head direction into angular bins.

The head direction space is divided into $B_\theta = 60$ bins and $P^{(k)} \in \mathbb{R}^{B_\theta}$ is calculated as
$$
P^{(k)}_{i} = \frac{\sum_t A_t^{(k)}~\overrightarrow{1}_{\theta_t \in \text{Bin}_i}}{\sum_t \overrightarrow{1}_{\theta_t \in \text{Bin}_i}}, \quad i \in \{1, \dots, B_\theta\}
$$
Where $A^{(k)}_t$ is the hidden layer’s activity (or value) of unit $k$ and $\theta$ is the agent's head direction.

**Note**: In this notebook, we use the terms _head direction_ and _theta_ interchangeably.

In [ ]:
def compute_polar_maps(
    hidden_states:np.array, head_directions:np.array, sigma:float,
    n_bins:int, limits=(-np.pi, np.pi)
):
    """
    Calculates the (smooth) polar map for each neuron based on the given hidden states and thetas.

    Args:
        hidden_states (np.array): (samples, n_hidden) Array of hidden states for each neuron.
        head_directions (np.array): (samples,) Array of head direction points.
        sigma (float, optional): Standard deviation of the circular filter. Defaults to 5.
        n_bins (float): Number of angular bins for the head directions.
        limits (tuple, optional): Limits of the hd space. Defaults to (-np.pi, np.pi).

    Returns:
        polar_maps (np.array): Array of polar maps for each neuron of shape (n_hidden, n_bins).
        occupancy (np.array): Occupancy map of shape (n_bins,).
    """
    
    occupancy = bin_data(head_directions, n_bins=n_bins, limits=limits)
    if sigma > 0:
        occupancy = gaussian_filter(occupancy, sigma, mode='wrap')

    n_cells = hidden_states.shape[1]
    polar_maps = np.empty(shape=(n_cells,occupancy.shape[0]))
    for i in range(n_cells):
        # YOUR CODE HERE (3)
        # calculate the activations for each cell, for each binned head direction
        activations = None
        
        # YOUR CODE HERE (4)
        # obtain the polar map for each cell by dividing the activations by the occupancy
        polar_maps[i] = None

    return polar_maps, occupancy


In [ ]:
N_BINS_POLAR = 60
bins_thet = np.linspace(-np.pi, np.pi, N_BINS_POLAR+1)
thetas_ticks = np.array(
    [np.mean([a, b]) for a, b in zip(bins_thet, bins_thet[1:])]
)
bins_thet = bins_thet[:-1]

In [ ]:
polar_maps, polar_occupancy = compute_polar_maps(
    hidden_states, head_directions, n_bins=N_BINS_POLAR, sigma=0.
)

polar_maps_smooth, polar_occupancy_smooth = compute_polar_maps(
    hidden_states, head_directions, n_bins=N_BINS_POLAR, sigma=2.5
)

print('Polar maps shape (n_hidden, n_bins):', polar_maps_smooth.shape)

Let's plot the polar map of the same neuron. Note that we will need to use the polar projection.

In [ ]:
idx = 9

fig, axs = plt.subplots(1, 2, figsize=(10, 4), subplot_kw={'projection': 'polar'})

axs[0].plot(
    np.append(thetas_ticks, thetas_ticks[0]), # we want to close the circle
    np.append(polar_maps[idx], polar_maps[idx][0]), # we want to close the circle
    lw=3, c='blue',
)
# axs[0].set_xticklabels([]) # remove degrees indication
axs[0].set_rticks([max(polar_maps[idx])], labels=[f"{max(polar_maps[idx]):.2f}"]) # add intensity indication
axs[0].tick_params(axis='y', labelsize=9)
axs[0].set_theta_direction(-1)
axs[0].set_theta_zero_location('N') # move 0 to the north
axs[0].grid(True)
axs[0].set_title(f'Un-smoothed Polar Map for Neuron {idx}')

axs[1].plot(
    np.append(thetas_ticks, thetas_ticks[0]), # we want to close the circle
    np.append(polar_maps_smooth[idx], polar_maps_smooth[idx][0]), # we want to close the circle
    lw=3, c='blue',
)
axs[1].set_rticks(
    [max(polar_maps_smooth[idx])],
    labels=[f"{max(polar_maps_smooth[idx]):.2f}"]
) # add intensity indication
axs[1].tick_params(axis='y', labelsize=9)
axs[1].set_theta_direction(-1)
axs[1].set_theta_zero_location('N') # move 0 to the north
axs[1].grid(True)
axs[1].set_title(f'Smoothed Polar Map for Neuron {idx}')

plt.show()

### 5. Spatial metrics

In this part we will examine some commonly used metrics used to quantify and classify spatial cells, following standard neuroscience classification criteria [2, 3]. We quantify spatial and directional selectivity using spatial information (SI) and resultant vector length (RVL), respectively. We compute these metrics on min-max normalised rate and polar maps.

**Spatial information (SI)** measures how much information (in bits) a cell's firing conveys about the agent's position per spike [4].

Given
- $B \quad\rightarrow$ number of bins

- $R_i \quad\rightarrow$ value of the rate map at bin $i$

- $p_i \quad\rightarrow$ probability of being in bin $i$ (occupancy probability)

- $\hat{R} = \sum_{i=1}^{B} p_i~R_i \quad\rightarrow$ overall mean activity

Then
$$
\text{SI}_{\text{sec}} = \sum_{i=1}^{B} p_i~R_i~\log_2 \left( \frac{R_i}{\hat{R}} \right)
$$
measured in **bits per second**. We can convert the SI in **bits per spike** dividing by the mean rate:
$$
\text{SI}_{\text{spike}} = \frac{\text{SI}_{\text{sec}}}{\hat{R}}
$$


In [ ]:
def spatial_information(rate_map: np.ndarray, occupancy: np.ndarray):
    """
    Calculates the Spatial Information (SI) for a given rate map.

    Args:
        rate_map (np.array): (n_bins, n_bins) Rate map of a single neuron.
        occupancy (np.array): occupancy map of shape (n_bins,).

    Returns:
        si (float): Spatial information content of the rate map.
    """
    # discard bins where occupancy is zero or NaN
    no_occ = np.logical_or(occupancy == 0, np.isnan(occupancy))
    _rate = rate_map[~no_occ]
    _occ = occupancy[~no_occ]

    # calculate total duration and occupancy probability
    duration = np.sum(_occ)
    _occ_prob = _occ / duration

    # calculate mean firing rate
    _rate_mean = np.sum(_rate*_occ) / duration

    mask = _rate > 0

    # YOUR CODE HERE (5)
    # use the equation above, calculate the SI (assuming we are measuring in bits per second)
    si = None


    return si

In [ ]:
rate_maps_min = np.moveaxis(
    np.tile(np.nanmin(rate_maps_smooth, axis=(1,2)), (N_BINS, N_BINS, 1)), -1, 0
)
rate_maps_max = np.moveaxis(
    np.tile(np.nanmax(rate_maps_smooth, axis=(1,2)), (N_BINS, N_BINS, 1)), -1, 0
)
rate_maps_norm = np.divide(
    (rate_maps_smooth - rate_maps_min),
    (rate_maps_max - rate_maps_min),
    where=np.logical_and(
        (rate_maps_max - rate_maps_min)!=0,
        ~np.isnan(rate_maps_smooth)
    ),
    out=rate_maps_smooth
)

# uniform_rate_map = np.ones_like(rate_maps_norm[0])/(n**2)
# if occ is None : occ = uniform_rate_map

si = np.array([spatial_information(rm, occupancy_smooth) for rm in rate_maps_norm])

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(si, bins=30, color='blue', alpha=0.7)
plt.title('Distribution of Spatial Information')
plt.xlabel('Spatial information')
plt.ylabel('Number of neurons')
plt.show()

**Resultant vector length (RVL)** measures the concentration of the directional preference.

Given
- $B \quad\rightarrow$ number of bins

- $P_i \quad\rightarrow$ value of the polar map at bin $i$

- $\mathbf{r} = \sum_{i=1}^B \exp\left(j\frac{2\pi}{B_\theta}~i\right)~P_i \quad\rightarrow$ resultant vector of polar map $P$, where $j$ is an imaginary unit.

Then
$$
\text{RVL} = \frac{|\mathbf{r}|}{\sum_{i=1}^B P_i}
$$

$\text{RVL}$ is a number between 0 and 1. 

If $\text{RVL}$ is close to 1, it indicates **ideal directional tuning** (all weights concentrated at a single angle). If it is close to 0, then directional tuning is **uniform** around the circle. Otherwise, some degrees of directional concentration is shown.

In [ ]:
def resultant_vector(polar_map, bins, spacing):
    """
    Calculate the resultant vector of a given set of angles.

    Args:
        polar_map (np.array): (n_bins,) Polar map of a single neuron.
        bins (np.array): Array of angles.
        spacing (float, optional): Known spacing between angles.
    Returns:
        rvl (float): Length of the resultant vector.
        rv_angle (float): Angle of the resultant vector.
    """

    # YOUR CODE HERE (6)
    # calculate resultant vector as sum of cos & sin angles
    rv = None
    
    # calculate the angle of the resultant vector
    rv_angle = np.angle(rv)

    # get length
    rvl = np.abs(rv) / np.sum(polar_map)

    # for data with known spacing, apply correction factor to correct
    # for bias in the estimation of r
    # (see Biostatistical Analysis, J. H. Zar, p. 601, equ. 26.16)
    rvl *= spacing / 2 / np.sin(spacing / 2)

    return rvl, rv_angle

In [ ]:
divisor = np.moveaxis(
    np.tile(polar_maps_smooth.sum(axis=1), (N_BINS_POLAR, 1)), -1, 0
)
polar_maps_norm = np.divide(
    polar_maps_smooth, divisor,
    where=divisor != 0, out=polar_maps_smooth
)

rv_tuple = [
    resultant_vector(
        polar_map=pm,
        bins=bins_thet,
        spacing=2*np.pi/N_BINS_POLAR
    ) for pm in polar_maps_norm
]

rvl = np.array([x[0] for x in rv_tuple])
rvangle = np.array([x[1] for x in rv_tuple])

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(rvl, bins=30, color='blue', alpha=0.7)
plt.xlabel('Resultant Vector Length')
plt.ylabel('Number of Neurons')
plt.title('Distribution of Resultant Vector Lengths')
plt.show()

### 6. Spatial neurons

Following standard neuroscience classification criteria [2, 3], we will identify **place cells** based on these two criteria:

* Spatial information > 0.3

* Place fields are contiguous (we define a place field here as a bin with mean firing rate greater than 95% quantile of that neuron)

We will classify **head directional cells** on whose RVL is greater than 0.35

In [ ]:
from utils import check_if_contiguous

# classify neurons as place cells based on spatial information
indices_place = np.where(si>0.3)[0]

# filter discontiguous place fields
for i in indices_place:
    place_fields = [tuple(x) for x in np.argwhere(rate_maps_norm[i] > np.quantile(rate_maps_norm[i], 0.95)).tolist()]
    if check_if_contiguous(place_fields, tolerance=1) is False:
        indices_place = np.delete(indices_place, np.where(indices_place == i))

print('Number of place cells:', len(indices_place))

In [ ]:
n_plot = 100
indices = np.random.choice(indices_place, size=n_plot, replace=False)

n_rows = int(np.ceil(n_plot/10))
fig, axs = plt.subplots(n_rows, 10, figsize=(12, n_rows*1.5))

for idx in range(n_plot):
    ax = axs.flat[idx]
    ax.imshow(rate_maps_smooth[indices[idx]], cmap='jet')
    ax.set_title(f'Neuron {indices[idx]}', fontsize='small')
    ax.set_axis_off()

plt.show()

In [ ]:
indices_hd = np.where(rvl>0.35)[0]

print('Number of head direction cells:', len(indices_hd))

In [ ]:
n_plot = 100
indices = np.random.choice(indices_hd, size=n_plot, replace=False)

n_rows = int(np.ceil(n_plot/10))
fig, axs = plt.subplots(n_rows, 10, figsize=(12, n_rows*1.5), subplot_kw={'projection': 'polar'})

for idx in range(n_plot):
    ax = axs.flat[idx]
    polar_map = polar_maps_smooth[indices[idx]]

    ax.plot(
        np.append(thetas_ticks, thetas_ticks[0]), # we want to close the circle
        np.append(polar_map, polar_map[0]), # we want to close the circle
        lw=3, c='blue',
    )
    ax.vlines(
        rvangle[indices[idx]], 0, np.max(polar_map),
        colors='red', lw=2
    )
    ax.set_rticks([])
    ax.set_xticklabels([])
    ax.tick_params(axis='y', labelsize=9)
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N') # move 0 to the north
    ax.grid(True)
    ax.set_title(f'Neuron {indices[idx]}', fontsize='small')

plt.show()

### 7. Applications

### References

[1] R. U. Muller and J. L. Kubie. The firing of hippocampal place cells predicts the future position of freely moving rats. Journal of Neuroscience, 9(12):4101–4110, 1989.

[2] J. O'Keefe and D. H. Conway. Hippocampal place units in the freely moving rat: why they fire where they fire. Experimental brain research, 31:573–590, 1978.

[3] J. C. Whittington, T. 390 H. Muller, S. Mark, G. Chen, C. Barry, N. Burgess, and T. E. Behrens. The tolman-eichenbaum machine: unifying space and relational memory through generalization in the hippocampal formation. Cell, 183(5):1249–1263, 2020.

[4] W. E. Skaggs, B. L. McNaughton, M. A. Wilson, and C. A. Barnes. Theta phase precession in hippocampal neuronal populations and the compression of temporal sequences. Hippocampus, 6(2):149–172, 1996.
